In [34]:
! pip install --upgrade pip setuptools wheel
! pip install --index https://pypi.voxel51.com fiftyone

In [ ]:
! pip install detecto
! pip install torch, torchvision

In [35]:
import fiftyone as fo
import fiftyone.brain as fob
import detecto
from detecto.core import Model
import torch
import torchvision

## Load trained  Model for Analysis

In [36]:
# Run the model on GPU if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
labels= ["House", "Car", "Ongoing Construction", "Built-up area", "Water Storage", "Bus", "Truck", "Train Track"]
# Load a pre-trained Faster R-CNN model
model = Model.load('/home/emmanuel/ammi_ml-proj/model.pth', labels)
print(model, ":Model ready")

<detecto.core.Model object at 0x7f446c290c18> :Model ready


## Load test dataset 

In [37]:
name = "object-detection_Project"
dataset_dir = "/home/emmanuel/ammi_ml-proj/dataset/"

# Create the dataset
dataset = fo.Dataset.from_dir(dataset_dir, fo.types.VOCDetectionDataset, name=name
)

 100% |██████████████████████████████████████████████████████████████████████████████████████████| 9/9 [87.2ms elapsed, 0s remaining, 103.2 samples/s]     


## Compute Dataset Uniqueness ( please change port on each run)

In [ ]:
fob.compute_uniqueness(dataset)
dups_view = dataset.sort_by("uniqueness")
# Launch the App
session = fo.launch_app(view=dups_view, port=7080)

## Select rondomly test images to use and make predictions on them

In [38]:
predictions_view = dataset.take(9, seed=45)

In [40]:
from PIL import Image
from detecto.utils import read_image
from torchvision.transforms import functional as func

In [41]:
# Add predictions to samples
with fo.ProgressBar() as pb:
    for sample in pb(predictions_view):
        # Load image
        image = Image.open(sample.filepath)
        image = func.to_tensor(image).to(device)
        c, h, w = image.shape

        # Perform inference
        image = read_image(sample.filepath)


        labels, boxes, scores = model.predict([image])[0]
        
        scores = scores.detach().numpy()
        boxes = boxes.detach().numpy()

        # Convert detections to FiftyOne format
        detections = []
        for label, score, box in zip(labels, scores, boxes):
            # Convert to [top-left-x, top-left-y, width, height]
            # in relative coordinates in [0, 1] x [0, 1]
            x1, y1, x2, y2 = box
            rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]

            detections.append(fo.Detection(
                label=label,
                bounding_box=rel_box,
                confidence=score
            ))

        # Save predictions to dataset
        sample["model"] = fo.Detections(detections=detections)
        sample.save()

print("Finished adding predictions")

 100% |██████████████████████████████████████████████████████████████████████████████████████████| 9/9 [35.8s elapsed, 0s remaining, 0.3 samples/s]       
Finished adding predictions


## Select predictions greater than 0.55 confidence 

In [42]:
from fiftyone import ViewField as F

# Only keep detections with confidence > 0.55
high_conf_view = predictions_view.filter_detections("model", F("confidence") > 0.55)

In [43]:
sample = high_conf_view.first()
print(sample.model.detections[0])

<Detection: {
    'id': '5fc62ac227871c3a7c5b6744',
    '_id': '5fc62ac227871c3a7c5b6744',
    'attributes': BaseDict({}),
    'label': 'House',
    'bounding_box': BaseList([
        0.1947205596499973,
        0.40360496662281176,
        0.13765078120761448,
        0.13736399897822626,
    ]),
    'mask': None,
    'confidence': 0.9840937256813049,
    'index': None,
    '_cls': 'Detection',
}>


In [44]:
print(predictions_view)

Dataset:        object-detection_Project
Media type:     image
Num samples:    9
Tags:           []
Sample fields:
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    model:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
Pipeline stages:
    1. Take(size=9, seed=45)


## View result to compare predictions and ground truth (please change port on each run)

In [45]:
! sudo sysctl kernel.unprivileged_userns_clone=1

# Open the dataset in the App
session = fo.launch_app(dataset=dataset, port=7050) 
# Load view containing the subset of samples for which we added predictions
session.view = predictions_view  

[sudo] password for emmanuel: 
App launched
